# Gerador de Músicas MIDI

O projeto consiste em realizar um estudo no qual serão geradas músicas sem intervenção humana. Para tal, utilizou-se de músicas clássicas de Chopin como dataset e um modelo de Deep Learning.

## Deep Learning - Uma breve introdução

O Deep Learning é um algoritmo de Machine Learning, no qual são utilizadas redes neurais que buscam "aprender" de forma similar ao comportamento humano.

Uma forma simples de entender é imaginar uma criança aprendendo a reconhecer objetos. A criança pode apontar para um objeto e dizer que é um carro. Dado isto, o pai/a mãe da criança pode reagir de duas maneiras: confirmar que o objeto que a criança apontou é um carro, ou falar "Não, isto é um jarro". Ao receber feedback suficiente, a criança começa a internalizar as características de cada objeto e cria um modelo mental que ajuda ela a reconhecer os diferentes objetos. Este modelo depende de uma comunicação efetiva entre os neurônios, transmitindo diferentes sinais e gerando este modelo complexo e hierárquico baseado no feedback recebido.

O Deep Learning busca replicar este comportamento, criando um modelo no qual não é necessário entender cada etapa e decisão feita devido à complexidade e profundidade deste. Para criar o modelo, são utilizadas múltiplas camadas de "neurônios digitais", que vão repassando o aprendizado de camada em camada.

Inicialmente, o modelo é alimentado com os dados a serem utilizados, e este tenta prever os dados, sem nenhuma intervenção. As previsões iniciais irão ser completamente (ou em sua maior parte) incorretas, mas conforme o modelo recebe feedback de suas previões, ele ajusta a comunicação entre seus "neurônios" até ser capaz de gerar previsões mais acuradas.

Existem diversos modelos de Deep Learning, e para o desenvolvimentod este projeto, irá ser utilizado o modelo de WaveNet.

## WaveNet

O WaveNet é um modelo de Deep Learning para áudios "crus" desenvolvido pelo Google DeepMind. Ele é chamado de "generative model", pois tem como objetivo gerar novos samples a partir da distribuição original dos dados. Ele atua de forma similar aos modelos de linguagem utilizados em NLP.

### Treinando o WaveNet

Para treinar o modelo de WaveNet, utiliza-se um trecho de uma onda crua de áudio (no caso, a onda de áudio no domínio do tempo) como input. Uma onda de áudio no domínio do tempo é representada na forma de diferente valores de amplitude em diferentes intervalos de tempo, como é possível visualizar no gif abaixo.

![Onda de áudio no domínio do tempo](https://jvbalen.github.io/figs/wavenet.gif)

A partir da sequência de valores de amplitude, o WaveNet tenta prever qual valor de amplitude vem em seguida.

In [ ]:
# System libraries
from pathlib import Path
from collections import Counter
from itertools import tee

# Numpy for arrays and matplotlib for notes histogram
import numpy as np
import matplotlib.pyplot as plt

# Music21 library for MIDI reading and creating
import music21 as m21
from music21.note import Note
from music21.chord import Chord

# sklearn to split train and test 
from sklearn.model_selection import train_test_split

In [ ]:
extracted_folder_path = "chopin/"
threaded = True # Esquenta o computador

In [ ]:
train_model = False

if train_model:
    # keras to train the model
    import keras.backend as kb
    import keras.callbacks as kc
    import keras.layers as kl
    import keras.models as km
else:
    # keras to load the model only
    from keras.models import load_model

In [ ]:
def read_midi(file):   
    notes = []
    
    # Parsing the MIDI file
    midi = m21.converter.parse(file)
  
    # Partition by instrument
    partition = m21.instrument.partitionByInstrument(midi)

    # Looping over all the instruments
    for part in partition.parts:
        # Select only the piano
        if 'Piano' not in str(part):
            continue

        # Checking if it is a note or a chord
        for element in part.recurse():
            if isinstance(element, Note):
                notes.append(str(element.pitch))

            elif isinstance(element, Chord):
                notes.append('.'.join(map(str, element.normalOrder)))

    return np.array(notes)

In [ ]:
files = (p for p in Path(extracted_folder_path).rglob("*") if p.is_file() and p.suffix == ".mid")

if threaded:
    from concurrent.futures import ProcessPoolExecutor

    with ProcessPoolExecutor() as pool:
        notes_list = list(pool.map(read_midi, files))
else:
    notes_list = list(map(read_midi, files))

In [ ]:
# Flattening notes_list
notes_f = np.concatenate(notes_list).ravel()

In [ ]:
# Get frequency of each note
unique_notes, counts = np.unique(notes_f, return_counts=True)

print(f"Number of unique notes: {len(unique_notes)}")

# Plot histogram
plt.figure(figsize=(5,5))
plt.hist(counts);

In [ ]:
# Getting the most frequent notes
frequent_notes = frozenset(unique_notes[counts >= 40])

In [ ]:
# Convert note to int and vice-versa

# Use tee to guarantee the iterators are the same
enum_it, rev_enum_it = tee(enumerate(frequent_notes))

# Helper functions
d_int_to_note = dict(enum_it)
int_to_note = lambda idx: d_int_to_note[idx]

d_note_to_int = {note: idx for idx, note in rev_enum_it}
note_to_int = lambda idx: d_note_to_int[idx]

In [ ]:
new_music = []

# Adding the most frequent notes
for notes in notes_list:
    new_music.append([d_note_to_int[note] for note in notes if note in frequent_notes])

In [ ]:
n_timesteps = 32
X = []
y = []

for note in new_music:
    for start in range(len(note) - n_timesteps):
        end = start + n_timesteps
        X.append(note[start:end])  # Input
        y.append(note[end])        # Output

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(np.array(X), np.array(y), test_size=0.3, random_state=42)

In [ ]:
if train_model:
    kb.clear_session()
    model = km.Sequential()

    # TODO: Validate x.unique() = frequent_notes
    model.add(kl.Embedding(len(frequent_notes), 100, input_length=32, trainable=True)) 

    model.add(kl.Conv1D(64, 3, padding='causal',activation='relu'))
    model.add(kl.Dropout(0.2))
    model.add(kl.MaxPool1D(2))

    model.add(kl.Conv1D(128, 3, activation='relu', dilation_rate=2, padding='causal'))
    model.add(kl.Dropout(0.2))
    model.add(kl.MaxPool1D(2))

    model.add(kl.Conv1D(256, 3, activation='relu', dilation_rate=4, padding='causal'))
    model.add(kl.Dropout(0.2))
    model.add(kl.MaxPool1D(2))

    model.add(kl.GlobalMaxPool1D())

    model.add(kl.Dense(256, activation='relu'))
    # TODO: Validate y.unique() = frequent_notes
    model.add(kl.Dense(len(frequent_notes), activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

    model.summary()

In [ ]:
if train_model:
    mc = kc.ModelCheckpoint('model.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)
    history = model.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_val, y_val), verbose=1, callbacks=[mc])
else:
    model = load_model('model.h5')

In [ ]:
idx = np.random.randint(0, X_val.shape[0] - 1)

random_music = X_val[idx]
predictions = []

for i in range(10):
    random_music = random_music.reshape(1, n_timesteps)

    prob  = model.predict(random_music)[0]
    y_pred = np.argmax(prob, axis=0)
    predictions.append(y_pred)

    random_music = np.insert(random_music[0], len(random_music[0]), y_pred)
    random_music = random_music[1:]

print(predictions)

In [ ]:
# Deleting previously generated MIDI file, if any
midi_file = "music.mid"
Path(midi_file).unlink(missing_ok=True)

In [ ]:
def convert_to_midi(prediction_output, mf):
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for offset, pattern in enumerate(prediction_output):
        # pattern is a chord
        if '.' in pattern or pattern.isdigit():
            notes = []
            
            for current_note in pattern.split('.'):
                new_note = Note(int(current_note))
                new_note.storedInstrument = m21.instrument.Piano()
                notes.append(new_note)
                
            new_chord = Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)

        # pattern is a note
        else:
            new_note = Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = m21.instrument.Piano()
            output_notes.append(new_note)

    midi_stream = m21.stream.Stream(output_notes)
    midi_stream.write('midi', fp=mf)

In [ ]:
# Creating MIDI file
convert_to_midi(map(int_to_note, predictions), midi_file)

# Shows MIDI in player
mf = m21.midi.MidiFile()
mf.open(midi_file)
mf.read()
mf.close()
s = m21.midi.translate.midiFileToStream(mf)
s.show('midi')